# Extraer un partido de Sofascore con `match_id` (ScraperFC)

Este cuaderno toma un `match_id` de Sofascore y utiliza **ScraperFC** para obtener:

- Información general del partido (equipos, marcador, fecha, estadio, etc.).
- Estadísticas del equipo.
- Estadísticas de jugadores.
- Posiciones promedio.
- Shotmap y momentum.

> Ajusta el `match_id` en la siguiente celda antes de ejecutar.


In [1]:
import pandas as pd
import ScraperFC as sfc
from pathlib import Path


## Configuración

Coloca tu `match_id` aquí. Puedes pegar directamente el ID o un `match_url` completo.


In [2]:
match_input = "15362081"  # <- Reemplaza con tu match_id o URL de Sofascore


In [3]:
def normalize_match_id(match_input: str) -> str:
    match_input = str(match_input).strip()
    if "#id:" in match_input:
        return match_input.split("#id:")[-1]
    return match_input

match_id = normalize_match_id(match_input)
match_id


'15362081'

## Inicializar ScraperFC


In [4]:
sofascore = sfc.Sofascore()


## Información general del partido


In [5]:
match_dict = sofascore.get_match_dict(match_id)

summary = {
    "match_id": match_id,
    "home_team": match_dict.get("homeTeam", {}).get("name"),
    "away_team": match_dict.get("awayTeam", {}).get("name"),
    "home_score": (match_dict.get("homeScore") or {}).get("display"),
    "away_score": (match_dict.get("awayScore") or {}).get("display"),
    "tournament": (match_dict.get("tournament") or {}).get("name"),
    "season": (match_dict.get("season") or {}).get("name"),
    "round": (match_dict.get("roundInfo") or {}).get("round"),
    "start_time": pd.to_datetime(match_dict.get("startTimestamp"), unit="s", errors="coerce"),
    "venue": (match_dict.get("venue") or {}).get("name"),
    "referee": (match_dict.get("referee") or {}).get("name"),
}

pd.DataFrame([summary])


Running


,match_id,home_team,away_team,home_score,away_score,tournament,season,round,start_time,venue,referee
0,15362081,Universidad Técnica de Cajamarca,Club Atlético Grau,2,0,"Liga 1, Apertura",Liga 1 2026,1,2026-01-30 20:15:00,Estadio Heroes de San Ramon,Junior Rivera


## Scraping de datos del partido


In [6]:
team_stats_df = sofascore.scrape_team_match_stats(match_id)
player_stats_df = sofascore.scrape_player_match_stats(match_id)
avg_positions_df = sofascore.scrape_player_average_positions(match_id)
shotmap_df = sofascore.scrape_match_shots(match_id)
momentum_df = sofascore.scrape_match_momentum(match_id)

{
    "team_stats": team_stats_df.shape,
    "player_stats": player_stats_df.shape,
    "avg_positions": avg_positions_df.shape,
    "shotmap": shotmap_df.shape,
    "momentum": momentum_df.shape,
}


{'team_stats': (119, 14),
 'player_stats': (40, 82),
 'avg_positions': (31, 15),
 'shotmap': (14, 20),
 'momentum': (92, 2)}

## ETL para Admin (Fantasy)

Este bloque transforma las estadísticas de ScraperFC al formato que espera el admin del Fantasy:
`player_id, match_id, goals, assists, minutesplayed, saves, fouls, yellow_cards, red_cards, clean_sheet, goals_conceded`.

- Usa columnas disponibles en `player_stats_df`.
- Completa `goals_conceded` a partir del marcador si no existe en el dataset.
- Calcula `clean_sheet` cuando corresponde (0/1) usando `goals_conceded` y minutos.


In [32]:
import numpy as np

output_dir = Path("matches_details")
output_dir.mkdir(parents=True, exist_ok=True)


def coalesce_column(df: pd.DataFrame, candidates: list[str]) -> pd.Series:
    for col in candidates:
        if col in df.columns:
            return df[col]
    return pd.Series([None] * len(df))

def normalize_position(value: object) -> str:
    if value is None or pd.isna(value):
        return ""
    text = str(value).strip().lower()
    if text in {"gk", "goalkeeper", "goalie", "portero"}:
        return "GK"
    if text in {"defender", "def", "df", "d", "defensa"}:
        return "D"
    if text in {"midfielder", "mid", "mf", "m", "mediocampista"}:
        return "M"
    if text in {"forward", "fw", "f", "delantero"}:
        return "F"
    return text.upper()

player_id = coalesce_column(player_stats_df, ["id", "player_id", "player.id"])
team_id = coalesce_column(player_stats_df, ["teamId", "team_id", "team.id"])
position = coalesce_column(player_stats_df, ["pos", "playerPosition", "positionName", "player.position"]).apply(normalize_position)
minutesPlayed = coalesce_column(player_stats_df, ["minutesPlayed", "minutesplayed", "minutes"])
goals = coalesce_column(player_stats_df, ["goals"])
assists = coalesce_column(player_stats_df, ["assists"])
saves = coalesce_column(player_stats_df, ["saves", "savesMade"])
fouls = coalesce_column(player_stats_df, ["fouls", "foulsCommitted"])

home_team_id = match_dict.get("homeTeam", {}).get("id")
away_team_id = match_dict.get("awayTeam", {}).get("id")
home_score = (match_dict.get("homeScore") or {}).get("display")
away_score = (match_dict.get("awayScore") or {}).get("display")

def _to_int(value, default=0):
    try:
        if value is None or (isinstance(value, float) and np.isnan(value)):
            return default
        return int(float(value))
    except Exception:
        return default

team_goals_conceded = {}
if home_team_id is not None and away_team_id is not None:
    team_goals_conceded[home_team_id] = _to_int(away_score, None)
    team_goals_conceded[away_team_id] = _to_int(home_score, None)

clean_sheet = []
goals_conceded_filled = []

for idx in range(len(player_stats_df)):
    team_value = team_id.iloc[idx] if len(team_id) > idx else None
    minutes_val = minutesPlayed.iloc[idx] if len(minutesPlayed) > idx else 0
    position_val = position.iloc[idx] if len(position) > idx else ""
    is_defensive = position_val in {"GK", "D"}

admin_stats_df = pd.DataFrame({
    "player_id": player_id,
    "match_id": match_id,
    "goals": goals,
    "assists": assists,
    "minutesPlayed": minutesPlayed,
    "saves": saves,
    "fouls": fouls,
})

admin_stats_df = admin_stats_df.fillna(0)
for col in ["player_id", "match_id", "goals", "assists", "minutesPlayed", "saves", "fouls"]:
    admin_stats_df[col] = pd.to_numeric(admin_stats_df[col], errors="coerce").fillna(0).astype(int)

admin_stats_df.head()


C:\Users\Alvaro\AppData\Local\Temp\ipykernel_46412\3110044294.py:73: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  admin_stats_df = admin_stats_df.fillna(0)


,player_id,match_id,goals,assists,minutesPlayed,saves,fouls
0,876932,15362081,0,0,90,2,0
1,876928,15362081,0,0,90,0,0
2,965761,15362081,0,0,73,0,1
3,874645,15362081,0,0,90,0,0
4,980889,15362081,0,0,90,0,3


### Texto para pegar en Admin

Copia el siguiente bloque y pégalo en el textarea de Admin (cada fila es un jugador).


In [33]:
admin_lines = admin_stats_df.apply(
    lambda row: ",".join(
        str(row[col])
        for col in [
            "player_id",
            "match_id",
            "goals",
            "assists",
            "minutesPlayed",
            "saves",
            "fouls",
        ]
    ),
    axis=1,
)
admin_payload = "\n".join(admin_lines)
admin_payload[:1000]  # preview


'876932,15362081,0,0,90,2,0\n876928,15362081,0,0,90,0,0\n965761,15362081,0,0,73,0,1\n874645,15362081,0,0,90,0,0\n980889,15362081,0,0,90,0,3\n883768,15362081,0,0,83,0,0\n975708,15362081,0,0,83,0,1\n879886,15362081,0,0,73,0,0\n846742,15362081,1,0,90,0,0\n869785,15362081,1,0,89,0,0\n143886,15362081,0,0,90,0,1\n957940,15362081,0,0,17,0,0\n123245,15362081,0,0,17,0,1\n944073,15362081,0,0,13,0,0\n1159653,15362081,0,0,13,0,1\n1466385,15362081,0,0,1,0,1\n924765,15362081,0,0,0,0,0\n1128886,15362081,0,0,0,0,0\n883743,15362081,0,0,0,0,0\n1176979,15362081,0,0,0,0,0\n805264,15362081,0,0,90,2,0\n830660,15362081,0,0,90,0,0\n384260,15362081,0,0,90,0,3\n845431,15362081,0,0,90,0,1\n790018,15362081,0,0,82,0,0\n830888,15362081,0,0,66,0,0\n805065,15362081,0,0,90,0,0\n931776,15362081,0,0,90,0,1\n888377,15362081,0,0,45,0,0\n1120902,15362081,0,0,66,0,0\n107625,15362081,0,0,90,0,1\n975540,15362081,0,0,45,0,1\n2044305,15362081,0,0,24,0,0\n1173448,15362081,0,0,24,0,0\n1650560,15362081,0,0,8,0,0\n922877,15362081,0

In [34]:
admin_stats_df.to_csv(output_dir / f"Fantasy_{match_id}.csv", index=False)


### Vista rápida de los datos


In [35]:
team_stats_df.head()


,name,home,away,compareCode,statisticsType,valueType,homeValue,awayValue,renderType,key,period,group,homeTotal,awayTotal
0,Ball possession,45%,55%,2,positive,event,45.00,55.00,2,ballPossession,ALL,Match overview,NaN,NaN
1,Expected goals,0.34,0.28,1,positive,event,0.34,0.28,1,expectedGoals,ALL,Match overview,NaN,NaN
2,Big chances,0,1,2,positive,event,0.00,1.00,1,bigChanceCreated,ALL,Match overview,NaN,NaN
3,Total shots,7,7,3,positive,event,7.00,7.00,1,totalShotsOnGoal,ALL,Match overview,NaN,NaN
4,Goalkeeper saves,2,2,3,positive,event,2.00,2.00,1,goalkeeperSaves,ALL,Match overview,NaN,NaN


In [36]:
player_stats_df.head()


,name,slug,shortName,position,jerseyNumber,height,userCount,gender,id,country,...,dispossessed,totalOffside,wonContest,goals,totalKeeperSweeper,accurateKeeperSweeper,errorLeadToAShot,bigChanceCreated,bigChanceMissed,teamName
0,Ángelo Campos,angelo-campos,Á. Campos,G,1,182.0,195,M,876932,"{'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Per...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Universidad Técnica de Cajamarca
1,Luis Garro,luis-garro,L. Garro,D,17,176.0,12,M,876928,"{'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Per...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Universidad Técnica de Cajamarca
2,Piero Serra,piero-serra,P. Serra,D,27,175.0,6,M,965761,"{'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Per...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Universidad Técnica de Cajamarca
3,Bruno Duarte,bruno-duarte,B. Duarte,D,2,190.0,23,M,874645,"{'alpha2': 'AR', 'alpha3': 'ARG', 'name': 'Arg...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Universidad Técnica de Cajamarca
4,Dylan Caro,dylan-caro,D. Caro,D,13,170.0,16,M,980889,"{'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Per...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Universidad Técnica de Cajamarca


## Guardar a Excel (opcional)


In [ ]:
output_dir = Path("matches_details")
output_dir.mkdir(parents=True, exist_ok=True)

output_file = output_dir / f"Sofascore_{match_id}.xlsx"
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    team_stats_df.to_excel(writer, sheet_name="Team Stats", index=False)
    player_stats_df.to_excel(writer, sheet_name="Player Stats", index=False)
    avg_positions_df.to_excel(writer, sheet_name="Average Positions", index=False)
    shotmap_df.to_excel(writer, sheet_name="Shotmap", index=False)
    momentum_df.to_excel(writer, sheet_name="Match Momentum", index=False)

output_file
